<a href="https://colab.research.google.com/github/CESARIUX2596/CS-Master/blob/master/Artificial_Vision_FInal_Proyect_Data_Augmentation_Aproach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guitar classification
In this project we will create a model that classifies guitar by body type in realtime using the videofeed of a camera.
The body types available to clasify are the following:


*   Acoustic
*   Double Cut
*   Les Paul
*   Telecaster
*   Strat
*   Ukulele


In this Aproach we will use data augmentation to provide more training information to the Convolutional Neuronal Network while training.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Mount Google drive as a device
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
'''
The folder structure contains the Train and Test directories
Each diractory has subfolders for each class that the classificatior will accept
'''
# Mount the folder with the dataset
os.chdir("/content/drive/My Drive/datasets/")
!ls

caos_guitar_dataset  Model1  tacos  TTY_dataset


In [4]:
base_directory =  'caos_guitar_dataset'
train_directory = os.path.join(base_directory, 'Train')
test_directory = os.path.join(base_directory, 'Test')

In [5]:
# Training directories
train_acoustic = os.path.join(train_directory, 'Acoustic')
train_double_cut = os.path.join(train_directory, 'Double_Cut')
train_les_paul = os.path.join(train_directory, 'Les_Paul')
train_strat = os.path.join(train_directory, 'Strat')
train_tele = os.path.join(train_directory, 'Telecaster')
train_ukulele = os.path.join(train_directory, 'Ukulele')

# Test directories
test_acoustic = os.path.join(test_directory, 'Acoustic')
test_double_cut = os.path.join(test_directory, 'Double_Cut')
test_les_paul = os.path.join(test_directory, 'Les_Paul')
test_strat = os.path.join(test_directory, 'Strat')
test_tele = os.path.join(test_directory, 'Telecaster')
test_ukulele = os.path.join(test_directory, 'Ukulele')

In [6]:
# Simple Convolutional Neuronal Network 
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150,3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(6, activation='sigmoid')
])

In [7]:
# LEARNING_RATE = 1e-4
# We use this lost function because the labels are probided as One Hot Encoding
LOSS = 'CategoricalCrossentropy' 
model.compile(loss=LOSS,
              optimizer=Adagrad(lr=0.01, epsilon=1e-6),
              metrics=['accuracy'])

In [8]:
# Create Data Generator for the image Augmentation
train_datagen = ImageDataGenerator(
      
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.20,
      height_shift_range=0.2,
      shear_range=0.20,
      zoom_range=0.3,
      horizontal_flip=True,
      fill_mode='reflect')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_directory,  # This is the source directory for training images
        color_mode='rgb',
        target_size=(150, 150),  # All images will be resized to 128x128
        batch_size=40,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        test_directory,
        color_mode='rgb',
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

# history = model.fit(
#       train_generator,
#       steps_per_epoch=108,  # 2000 images = batch_size * steps
#       epochs=150,
#       validation_data=validation_generator,
#       validation_steps=54,  # 1000 images = batch_size * steps
#       verbose=2)
      

Found 4320 images belonging to 6 classes.
Found 1080 images belonging to 6 classes.


In [ ]:
# model.save('Model1')

INFO:tensorflow:Assets written to: Model1/assets


In [9]:
reconstructed_model = keras.models.load_model("Model1")


In [10]:
# Continue training model to get 210 epochs
history = reconstructed_model.fit(
      train_generator,
      steps_per_epoch=108,  # 2000 images = batch_size * steps
      epochs=60,
      validation_data=validation_generator,
      validation_steps=54,  # 1000 images = batch_size * steps
      verbose=2)

Epoch 1/60


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


108/108 - 2038s - loss: 1.7103 - accuracy: 0.3097 - val_loss: 1.3169 - val_accuracy: 0.4556
Epoch 2/60
108/108 - 174s - loss: 1.1834 - accuracy: 0.5470 - val_loss: 1.0405 - val_accuracy: 0.5991
Epoch 3/60
108/108 - 168s - loss: 1.0459 - accuracy: 0.5933 - val_loss: 1.0567 - val_accuracy: 0.6185
Epoch 4/60
108/108 - 169s - loss: 0.9974 - accuracy: 0.6211 - val_loss: 1.0349 - val_accuracy: 0.6222
Epoch 5/60
108/108 - 168s - loss: 0.9939 - accuracy: 0.6150 - val_loss: 1.0135 - val_accuracy: 0.6269
Epoch 6/60
108/108 - 167s - loss: 0.9418 - accuracy: 0.6419 - val_loss: 1.2207 - val_accuracy: 0.5454
Epoch 7/60
108/108 - 167s - loss: 0.9620 - accuracy: 0.6294 - val_loss: 0.9493 - val_accuracy: 0.6389
Epoch 8/60
108/108 - 165s - loss: 0.9291 - accuracy: 0.6400 - val_loss: 1.0621 - val_accuracy: 0.6028
Epoch 9/60
108/108 - 167s - loss: 0.9330 - accuracy: 0.6421 - val_loss: 0.9625 - val_accuracy: 0.6463
Epoch 10/60
108/108 - 168s - loss: 0.9383 - accuracy: 0.6370 - val_loss: 0.9947 - val_accura

In [11]:
reconstructed_model.save('Model1_Extended_210Epochs')

INFO:tensorflow:Assets written to: Model1_Extended_210Epochs/assets
